In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, KFold, cross_val_score
import matplotlib.pyplot as plt

In [ ]:
df_bmkt_original = pd.read_csv('https://raw.githubusercontent.com/hcimwtc/ML2022/main/bank.csv',sep=';')
df_bmkt_original = df_bmkt_original.rename(columns={'y': 'Deposit'})
df_bmkt = df_bmkt_original.copy()
df_bmkt_original

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,Deposit
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,33,services,married,secondary,no,-333,yes,no,cellular,30,jul,329,5,-1,0,unknown,no
4517,57,self-employed,married,tertiary,yes,-3313,yes,yes,unknown,9,may,153,1,-1,0,unknown,no
4518,57,technician,married,secondary,no,295,no,no,cellular,19,aug,151,11,-1,0,unknown,no
4519,28,blue-collar,married,secondary,no,1137,no,no,cellular,6,feb,129,4,211,3,other,no


INSPECTION STEP
This step is to check the detail of the dataset, try to understand each attributes' meaning and the relationship between each other and the target before moving on to pre-processing.

In [ ]:
df_bmkt.describe()

,age,balance,day,duration,campaign,pdays,previous
count,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000
mean,41.170095,1422.657819,15.915284,263.961292,2.793630,39.766645,0.542579
std,10.576211,3009.638142,8.247667,259.856633,3.109807,100.121124,1.693562
min,19.000000,-3313.000000,1.000000,4.000000,1.000000,-1.000000,0.000000
25%,33.000000,69.000000,9.000000,104.000000,1.000000,-1.000000,0.000000
50%,39.000000,444.000000,16.000000,185.000000,2.000000,-1.000000,0.000000
75%,49.000000,1480.000000,21.000000,329.000000,3.000000,-1.000000,0.000000
max,87.000000,71188.000000,31.000000,3025.000000,50.000000,871.000000,25.000000


In [ ]:
# inspect categorical data
for col in df_bmkt:
    if df_bmkt[col].dtype == object:
        print('Column: \"', col, '\"\n', df_bmkt[col].value_counts(), '\n', sep='')

Column: "job"
management       969
blue-collar      946
technician       768
admin.           478
services         417
retired          230
self-employed    183
entrepreneur     168
unemployed       128
housemaid        112
student           84
unknown           38
Name: job, dtype: int64

Column: "marital"
married     2797
single      1196
divorced     528
Name: marital, dtype: int64

Column: "education"
secondary    2306
tertiary     1350
primary       678
unknown       187
Name: education, dtype: int64

Column: "default"
no     4445
yes      76
Name: default, dtype: int64

Column: "housing"
yes    2559
no     1962
Name: housing, dtype: int64

Column: "loan"
no     3830
yes     691
Name: loan, dtype: int64

Column: "contact"
cellular     2896
unknown      1324
telephone     301
Name: contact, dtype: int64

Column: "month"
may    1398
jul     706
aug     633
jun     531
nov     389
apr     293
feb     222
jan     148
oct      80
sep      52
mar      49
dec      20
Name: month, dtype: 

In [ ]:
df_bmkt.value_counts('job')

job
management       969
blue-collar      946
technician       768
admin.           478
services         417
retired          230
self-employed    183
entrepreneur     168
unemployed       128
housemaid        112
student           84
unknown           38
dtype: int64

In [ ]:
# evaluate the correlation between columns
df_bmkt.apply(lambda x: x.factorize()[0]).corr(method='kendall')

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,Deposit
age,1.000000,0.078246,-0.015937,-0.041327,-0.003711,0.012105,-0.081073,0.004750,0.021018,0.016568,0.019199,0.008438,-0.009106,0.011356,0.008491,0.012585,0.043120
job,0.078246,1.000000,-0.015812,-0.107979,0.001818,-0.012864,-0.099110,0.011512,0.003684,-0.000018,0.031764,-0.020920,-0.010936,0.025052,0.026062,0.028287,0.041276
marital,-0.015937,-0.015812,1.000000,0.105791,0.044004,-0.014412,-0.032202,-0.022582,-0.040922,0.019401,-0.011827,0.000529,-0.030696,0.009262,0.008641,0.007916,0.061947
education,-0.041327,-0.107979,0.105791,1.000000,-0.011827,0.006916,-0.085739,-0.048969,-0.120743,-0.008954,0.067710,-0.009116,-0.003301,0.035339,0.038552,0.039428,0.045380
default,-0.003711,0.001818,0.044004,-0.011827,1.000000,-0.018621,0.006881,0.063994,-0.001592,0.008787,0.011335,-0.032355,-0.003590,-0.036727,-0.037183,-0.036782,0.001303
balance,0.012105,-0.012864,-0.014412,0.006916,-0.018621,1.000000,0.024842,-0.012531,0.015039,-0.004271,-0.012933,0.098109,-0.020644,0.013842,0.000606,0.002879,0.027727
housing,-0.081073,-0.099110,-0.032202,-0.085739,0.006881,0.024842,1.000000,0.018451,0.134117,-0.039011,-0.273634,0.020400,-0.026954,0.061322,0.059975,0.051916,-0.104683
loan,0.004750,0.011512,-0.022582,-0.048969,0.063994,-0.012531,0.018451,1.000000,-0.008199,0.038319,0.072504,-0.010321,-0.002610,-0.037021,-0.038145,-0.043235,-0.070517
contact,0.021018,0.003684,-0.040922,-0.120743,-0.001592,0.015039,0.134117,-0.008199,1.000000,-0.016194,-0.291057,0.019863,0.019006,-0.205274,-0.206770,-0.209378,-0.101799
day,0.016568,-0.000018,0.019401,-0.008954,0.008787,-0.004271,-0.039011,0.038319,-0.016194,1.000000,0.049936,-0.000990,0.020254,-0.033040,-0.033903,-0.031821,0.020060


In [ ]:
# rank the correlation of columns in respect to the target ('y')
df_corr = df_bmkt.apply(lambda x: x.factorize()[0]).corr(method='kendall')
df_corr['Deposit'].abs().sort_values()

default      0.001303
day          0.020060
balance      0.027727
month        0.030415
job          0.041276
age          0.043120
education    0.045380
marital      0.061947
campaign     0.063450
loan         0.070517
contact      0.101799
housing      0.104683
duration     0.141314
pdays        0.152364
previous     0.156751
poutcome     0.176707
Deposit      1.000000
Name: Deposit, dtype: float64

# DATA PRE-PROCESSING

Similar to the Adult dataset, this dataset has over 15 attributes and 4000+ instances, to be able to perform better accuracy, some irrelavant columns is drooped.

- Contact which is not useful to the prediction
- Month/Day/Campaign/Previous : I am not quite sre what they actually mean but according to correlation and figures, they do not seems to be very helpful toward the prediction. 

Hence, these columns are dropped to handle the data better and hopefully will be able to increase the accuracy.


In [ ]:
# drop columns that are duplicate, irrelevant, or too imbalanced
df_bmkt.drop(columns = ['contact', 'month', 'day', 'campaign', 'previous'], axis = 1, inplace=True)

In [ ]:
#Perform Standard Encoding and OneHot Encoding

# Create a dummy dataframe so axis will still exist after the encoding process
df_bmkt_ud = df_bmkt.copy()

# sort numerical features for encoding
scaler = StandardScaler()
numeric_feats = ['age', 'balance', 'duration', 'pdays']
df_bmkt_ud[numeric_feats] = scaler.fit_transform(df_bmkt_ud[numeric_feats])

df_bmkt_ud.head()

encoder = OneHotEncoder(sparse=False)
categorical_feats = ['job', 'marital', 'education', 'default', 'poutcome']

#Sort categorical features for encoding
df_bmkt_enc = pd.DataFrame(encoder.fit_transform(df_bmkt_ud[categorical_feats]))
df_bmkt_enc.columns = encoder.get_feature_names_out(categorical_feats)

# Replace Categotical Data with Encoded Data
df_bmkt_ud = df_bmkt_ud.drop(categorical_feats ,axis=1)
df_bmkt_ud = pd.concat([df_bmkt_enc, df_bmkt_ud], axis=1)

# Encode the "Y/Deposit" vale from Yes/No to 0/1
df_bmkt_ud['Deposit'] = df_bmkt_ud['Deposit'].apply(lambda x: 1 if x == 'yes' else 0)

print(df_bmkt_ud.head())

   job_admin.  job_blue-collar  job_entrepreneur  job_housemaid  \
0         0.0              0.0               0.0            0.0   
1         0.0              0.0               0.0            0.0   
2         0.0              0.0               0.0            0.0   
3         0.0              0.0               0.0            0.0   
4         0.0              1.0               0.0            0.0   

   job_management  job_retired  job_self-employed  job_services  job_student  \
0             0.0          0.0                0.0           0.0          0.0   
1             0.0          0.0                0.0           1.0          0.0   
2             1.0          0.0                0.0           0.0          0.0   
3             1.0          0.0                0.0           0.0          0.0   
4             0.0          0.0                0.0           0.0          0.0   

   job_technician  ...  poutcome_other  poutcome_success  poutcome_unknown  \
0             0.0  ...             0.0

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


# '''MODEL TRAINING & CROSS VALIDATION'''

In [ ]:
# define new X for train/test splitting
X = df_bmkt.drop('Deposit', axis=1)
y = df_bmkt['Deposit']

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X , y, test_size=0.2, random_state=1)

# '''DECISION TREE'''

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import confusion_matrix, f1_score
# train/test split with KFold (K=2)
kf = KFold(n_splits=2)
DT = DecisionTreeClassifier(random_state=77)
scoreDT = []

# start training and evaluate score
count = 0
for (train_idx, test_idx) in kf.split(X, y):
    count += 1
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    DT.fit(X_train, y_train) 
    scoreDT.append(DT.score(X_test, y_test))
    y_pred = DT.predict(X_test)
    f = f1_score(y_true = y_test, y_pred = y_pred, average = 'weighted')
    print('f1 #', count, ': ', f, sep='')
    
print('Best:', max(scoreDT))
print('Worst:', min(scoreDT))
print('Overall:', np.mean(scoreDT))

f1 #1: 0.8680181213759741
f1 #2: 0.8689415918421295
Best: 0.8690844758956214
Worst: 0.8676991150442478
Overall: 0.8683917954699346


In [ ]:
# score by cross-validation with default function
print('Cross-validation Score:', cross_val_score(DT, X, y, cv=2))

Cross-validation Score: [0.86554622 0.86725664]


# '''NEURAL NETWORK'''

In [ ]:
# create a definition to train and eval score for NN model
def NN_score(hid, lr, mmt):
    scoreNN = []
    NN = MLPClassifier(hidden_layer_sizes = (hid,), learning_rate_init = lr, momentum = mmt, random_state = 77)
    
    count = 0
    for (train_idx, test_idx) in kf.split(X, y):
        count += 1
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        NN.fit(X_train, y_train) 
        scoreNN.append(NN.score(X_test, y_test))
        y_pred = NN.predict(X_test)
        f = f1_score(y_true = y_test, y_pred = y_pred, average = 'weighted')
        print('f1 #', count, ': ', f, sep='')
        
    print('Best:', max(scoreNN))
    print('Worst:', min(scoreNN))
    print('Overall:', np.mean(scoreNN))

In [ ]:
# train NN with hidden-layer = 2, learning-rate = 0.1, momentum = 0.9
NN_score(2, 0.01, 0.9)

f1 #1: 0.824032766544199
f1 #2: 0.8373090713145841
Best: 0.8893805309734514
Worst: 0.880141530296329
Overall: 0.8847610306348902


In [ ]:
#Adjusting the parameter with hidden-layer = 3, learning-rate = 0.1, momentum = 0.9
NN_score(3, 0.01, 0.9)

f1 #1: 0.824032766544199
f1 #2: 0.8373090713145841
Best: 0.8893805309734514
Worst: 0.880141530296329
Overall: 0.8847610306348902


In [ ]:
##Adjusting the parameter with hidden-layer = 3, learning-rate = 0.007, momentum = 0.9
NN_score(3, 0.007, 0.9)

f1 #1: 0.824032766544199
f1 #2: 0.8373090713145841
Best: 0.8893805309734514
Worst: 0.880141530296329
Overall: 0.8847610306348902


# Conclusion

Based on the result, Decision Tree perform better than some of the ANN result in this dataset, thought ANN has an overall 0.88 accuracy, the F1 and F2, CV score of Decision Tree are better than ANN. This show that even Neural Network is a better algorithm than Decision Tree, there is still moment that Decision Tree outperform NN.